In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
from extractor.forex_extractor import FOREXExtractor
from time import sleep

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
prices = market.retrieve("prices")
market.store("stocks",prices)
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("stocks")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2023-05-20 2023-05-27


In [4]:
market.connect()
data = FOREXExtractor.extract(start,end)
values = pd.DataFrame(data["rates"].values.tolist())
dates = data["rates"].keys().tolist()
values["dates"]= dates
market.store("forex",values)
market.disconnect()

In [5]:
values

,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,dates
0,3.672304,87.337737,103.201367,386.420403,1.797385,540.503981,231.503026,1.503533,1.8020,1.703970,...,2.70255,0.741535,604.503597,118.730364,250.303593,19.451604,9001.203589,18.849725,321.999592,2023-05-20
1,3.672145,87.337737,103.201367,386.419718,1.797385,540.503940,231.544797,1.501727,1.8020,1.702909,...,2.70255,0.741535,604.492858,118.729872,250.297835,19.426250,9001.225372,18.849725,321.999592,2023-05-21
2,3.672198,87.999913,102.650135,386.910286,1.803113,540.530447,234.327797,1.503910,1.8000,1.698714,...,2.70255,0.748890,606.496520,110.629410,250.298985,19.263502,9001.202279,18.883676,321.999592,2023-05-22
3,3.672306,86.501353,103.149717,386.249721,1.801646,540.505905,234.917983,1.512120,1.8025,1.691746,...,2.70255,0.747762,607.498067,111.250401,250.303487,19.201798,9001.200468,19.117579,321.999592,2023-05-23
4,3.672200,87.503281,102.903129,385.939944,1.799875,540.496467,235.740970,1.527940,1.8000,1.699323,...,2.70255,0.748781,608.501776,111.374954,250.350243,19.242103,9001.200890,19.149618,321.999592,2023-05-24
5,3.672198,87.500934,102.896439,386.397012,1.801688,545.498782,235.735501,1.538905,1.8000,1.697666,...,2.70255,0.749470,608.503029,111.374974,250.350186,19.789605,9001.186468,19.347289,321.999592,2023-05-25
6,3.672204,87.000368,103.203989,386.520403,1.802099,552.503981,235.614184,1.534204,1.8025,1.703970,...,2.70255,0.746965,610.503597,111.650364,250.350363,19.659890,9001.203589,19.273756,321.999592,2023-05-26
7,3.672204,87.000368,103.203989,386.520403,1.802099,552.503981,235.614184,1.534204,1.8025,1.703970,...,2.70255,0.746965,610.503597,111.650364,250.350363,19.659890,9001.203589,19.273756,321.999592,2023-05-27


In [6]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        try:
            data = TiingoExtractor.extract(ticker,start,end)
            data["ticker"] = ticker
            market.store("stocks",data)
        except:
            print(ticker,"tiingo")
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

  1%|█▏                                                                                                                      | 5/490 [00:03<05:15,  1.54it/s]

market prices documents must be a non-empty list


 73%|██████████████████████████████████████████████████████████████████████████████████████▏                               | 358/490 [03:37<01:19,  1.67it/s]

market prices documents must be a non-empty list


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▏           | 441/490 [04:28<00:30,  1.62it/s]

market prices documents must be a non-empty list


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [04:57<00:00,  1.65it/s]


In [7]:
data

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,ticker
0,2023-05-22T00:00:00.000Z,179.52,181.415,178.26,178.86,1911378,179.52,181.415,178.26,178.86,1911378,0.0,1.0,ZTS
1,2023-05-23T00:00:00.000Z,172.67,177.980,171.40,177.88,2301764,172.67,177.980,171.40,177.88,2301764,0.0,1.0,ZTS
2,2023-05-24T00:00:00.000Z,169.14,171.385,168.52,171.14,2437856,169.14,171.385,168.52,171.14,2437856,0.0,1.0,ZTS
3,2023-05-25T00:00:00.000Z,165.24,169.970,163.57,169.63,3626334,165.24,169.970,163.57,169.63,3626334,0.0,1.0,ZTS
4,2023-05-26T00:00:00.000Z,164.74,166.630,164.35,165.66,1909477,164.74,166.630,164.35,165.66,1909477,0.0,1.0,ZTS


In [8]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:36<00:00, 13.40it/s]


In [9]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
75,0.479008,0.265008,0.273971,0.198063,0.115162,AVGO,Broadcom Inc.,Information Technology,Semiconductors
47,0.408899,0.015560,0.063624,0.171837,0.090589,ANET,Arista Networks,Information Technology,Communications Equipment
144,-0.027216,0.059687,0.020221,0.080647,0.072327,DLR,Digital Realty,Real Estate,Specialized REITs
312,0.521990,0.056697,0.104252,0.161731,0.066760,MPWR,Monolithic Power Systems,Information Technology,Semiconductors
428,0.208162,-0.028933,0.123619,0.061542,0.066207,TER,Teradyne,Information Technology,Semiconductor Equipment
196,0.060647,-0.034379,0.002488,0.039553,0.062390,F,Ford Motor Company,Consumer Discretionary,Automobile Manufacturers
304,0.470621,0.242664,0.185726,0.116262,0.062060,MU,Micron Technology,Information Technology,Semiconductors
265,0.228802,0.155894,0.199672,0.095958,0.062038,KLAC,KLA Corporation,Information Technology,Semiconductor Equipment
377,0.122280,-0.008228,0.077553,0.040782,0.060982,QRVO,Qorvo,Information Technology,Semiconductors
379,0.035627,-0.115360,-0.047393,0.048157,0.060854,QCOM,Qualcomm,Information Technology,Semiconductors


In [10]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Information Technology,0.159531,0.015108,0.052227,0.022080,0.027426
Consumer Staples,0.002039,-0.018336,-0.056061,-0.015293,0.000735
Industrials,0.037184,-0.019739,-0.017349,-0.013596,0.007779
Consumer Discretionary,0.064245,-0.027270,-0.036998,-0.020269,0.007701
Communication Services,0.064514,-0.027546,-0.015200,-0.014655,0.018119
Financials,-0.095129,-0.033449,-0.032206,-0.014583,0.006844
Health Care,-0.029710,-0.037309,-0.063662,-0.035493,-0.000327
Real Estate,-0.057706,-0.042988,-0.057422,-0.024428,0.010003
Utilities,-0.068878,-0.045495,-0.069908,-0.026302,-0.001547
